# **Image Dataset**

## Importação de pacotes e leitura dos dados

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


### Função auxiliar para separação dos conjuntos de dados

In [2]:
def get_features_target_data(df, indexes_path):

    indexes = pd.read_csv(indexes_path)

    filtered_df = df.filter(items = indexes.sample_index, axis=0)

    X = filtered_df.drop(columns= 'label')
    y = filtered_df['label']

    return X, y

def get_train_val_set_data(df):

    X_train, y_train = get_features_target_data(df, '../dataset/Data/train_indexes.csv')
    X_val, y_val = get_features_target_data(df, '../dataset/Data/val_indexes.csv')
    X_test, y_test = get_features_target_data(df, '../dataset/Data/test_indexes.csv')

    return X_train, y_train, X_val, y_val, X_test, y_test

### Leitura dos dados e separação em treino, validação e test

In [3]:
dataset = pd.read_csv('../dataset/Data/features_30_sec.csv')

In [4]:
X_train, y_train, X_val, y_val, X_test, y_test = get_train_val_set_data(dataset)

In [5]:
X_train.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
823,reggae.00023.wav,661794,0.428635,0.089221,0.107873,0.002750,2354.780333,738663.782474,2336.461430,135041.508993,...,9.143028,55.996277,-7.756232,54.388203,0.631764,61.935009,-4.122077,67.025398,0.496620,110.006645
241,country.00041.wav,661794,0.397855,0.082284,0.113301,0.000587,2634.080774,820747.026509,2898.541576,233608.971679,...,0.558818,42.996780,-2.325614,41.673706,-4.613111,41.339603,-0.618727,29.171967,-4.939899,29.804676
181,classical.00081.wav,661794,0.266827,0.084227,0.016546,0.000050,1213.971712,83056.685521,1539.592686,54396.267455,...,-2.286594,46.745155,-3.124706,54.713120,-0.213338,67.064110,2.632269,75.738228,1.816808,102.780624
103,classical.00003.wav,661794,0.234229,0.083451,0.029442,0.000357,1526.629882,91344.309888,1499.439342,60196.227560,...,1.499585,125.197098,0.597389,111.539612,-0.595684,151.719910,2.341259,133.902283,2.640536,210.365616
456,hiphop.00056.wav,661794,0.491658,0.086806,0.166640,0.001742,2355.729244,355112.573212,2497.062588,112732.414725,...,2.313654,34.634190,-2.811913,28.848858,1.763052,43.351460,0.856901,35.401611,-4.358317,49.211880


## Implementação do baseline

In [6]:
features = dataset.columns.drop(['filename','label'])
X_train = X_train.drop(columns= 'filename')
X_val = X_val.drop(columns= 'filename')
X_test = X_test.drop(columns= 'filename')

---
**PERGUNTA - É um problema desbalanceado?**: 

Para o problema proposto, nota-se que as classes no treino estão bem distribuídas entre si. Assim, não identifica-se um problema desbalanceado

In [7]:
y_train.value_counts(normalize = True)

classical    0.1
hiphop       0.1
pop          0.1
country      0.1
reggae       0.1
blues        0.1
disco        0.1
metal        0.1
rock         0.1
jazz         0.1
Name: label, dtype: float64

---
**PERGUNTA - Os dados precisarão de algum tipo de pré-processamento?**: 

No caso do arquivo csv será necessário realizar apenas a normalização pois nosso baseline trata-se de uma SVM. Para os arquivos de imagem, é necessário fazer a extração de cores por pixel. Por fim, no caso dos arquivos de áudio, é necessário convertê-los em alguma forma interpretável como uma sequência de inteiros por frame da música.

### 5. **Implementação do Baseline**

Como Baseline, optou-se pelo uso de uma SVM para classificação. No caso, não foram implementadas otimizações de hiperparâmetros. A escolha da SVM foi feita pois o número de amostras disponíveis não é alto e dito algoritmo é robusto a um alto número de features

In [8]:
from sklearn.svm import SVC
from sklearn import metrics

In [9]:
clfBaseline = SVC()

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

scaledXtrain = scaler.transform(X_train)
scaledXval = scaler.transform(X_val)

In [11]:
clfBaseline.fit(scaledXtrain, y_train)

SVC()

---
**PERGUNTA - Quais as métricas utilizadas neste problema? Acurácia, erro, MSRE, etc.? O que elas significam?**: 

Para o problema proposto é possível a aplicação de métricas como acurácia balanceada, recall, precisão e f1-score. O significado de cada uma é apresentado abaixo:

1.   Acurácia: Média de TP/(TP + TN + FP + FN) entre as classes
2.   Recall: TP / FN + TP
3.   Precisão: TP / TP + FP
4.   F1-Score: Média harmônica de recall e precisão



In [12]:
resultados = clfBaseline.predict(scaledXval)
print(metrics.classification_report(y_val,resultados))

              precision    recall  f1-score   support

       blues       0.79      0.73      0.76        15
   classical       1.00      1.00      1.00        15
     country       0.92      0.80      0.86        15
       disco       0.50      0.53      0.52        15
      hiphop       0.79      0.73      0.76        15
        jazz       0.82      0.93      0.87        15
       metal       0.69      0.73      0.71        15
         pop       0.92      0.73      0.81        15
      reggae       0.65      0.73      0.69        15
        rock       0.56      0.60      0.58        15

    accuracy                           0.75       150
   macro avg       0.76      0.75      0.76       150
weighted avg       0.76      0.75      0.76       150



In [13]:
print(metrics.balanced_accuracy_score(y_val,resultados))

0.7533333333333332


Avaliando os resultados acima, nota-se um resultado muito satisfatório para o Baseline. As métricas de acurácia, recall e precisão ficaram em torno de ~0.7 o que é muito positivo dado o grau de complexidade limitado para o modelo.